Что произойдет в результате выполнения следующей команды? <br>
import pandas as pnds

In [2]:
# Сможем использовать библиотеку pandas, обращаясь к ней как pnds

Перед тем, как приступать к работе, давайте загрузим данные. Предположим, что данные лежат в переменной path_to_file (path_to_file = "/some/path/to/file.csv"). Импортируйте библиотеку pandas, а затем загрузите данные. <br>
Результат сохраните в переменную taxi.

In [5]:
path_to_file = '2_taxi_nyc.csv'

import pandas as pd
taxi = pd.read_csv(path_to_file)

Проверьте, сколько всего строк и столбцов имеется в датасете.

In [6]:
taxi.shape

(29101, 14)

In [17]:
# 29101

Давайте посмотрим на типы колонок. Все ли из них считались правильно?

In [8]:
taxi.dtypes

pickup_dt        object
pickup_month     object
borough          object
pickups           int64
hday             object
spd             float64
vsb             float64
temp            float64
dewp            float64
slp             float64
pcp 01          float64
pcp 06          float64
pcp 24          float64
sd              float64
dtype: object

In [16]:
# float64

В названиях столбцов pcp 01, pcp 06, pcp 24 встречается пробел. Это не очень удобно, если в дальнейшем вы планируете обращаться к колонкам через точку, без использования кавычек и скобочек. 
<br>
Замените пробел в названиях на знак нижнего подчеркивания. 
<br>
Данные сохранены в переменную taxi. Данные должны быть изменены в этом же датафрейме.

In [9]:
taxi = taxi.rename(columns={
    "pcp 01":"pcp_01",
    "pcp 06":"pcp_06",
    "pcp 24":"pcp_24"
})

Вас попросили узнать, сколько записей (строк) в датафрейме относятся к району Манхэттен (Manhattan). Получить ответ на данный вопрос можно было бы, например, с помощью следующей команды: <br>
taxi.query("borough == 'Manhattan'").shape[0] <br>
А если мы хотим посмотреть, сколько раз встречается каждый из районов? Неужели придется использовать подобную конструкцию для каждого уровня переменной? Здесь на помощь приходит метод value_counts(), <br>
Сколько раз в данных встречается район Бруклин (Brooklyn)?

In [11]:
taxi.borough.value_counts()

borough
Bronx            4343
Brooklyn         4343
EWR              4343
Manhattan        4343
Queens           4343
Staten Island    4343
Name: count, dtype: int64

In [15]:
# 4343

Следующая задача: выяснить, из какого района было совершено наибольшее количество поездок за весь период. 

Для начала, посчитайте общее количество поездок (pickups), без группировки.

In [12]:
taxi.pickups.sum()

14265773

А теперь сгруппируйте данные по району (borough) и укажите, из какого пункта было совершено больше всего поездок.

In [13]:
taxi.groupby('borough').agg({'pickups':'sum'})

,pickups
borough,
Bronx,220047
Brooklyn,2321035
EWR,105
Manhattan,10367841
Queens,1343528
Staten Island,6957


In [14]:
# Manhattan

Возможно, в предыдущем степе вы просто посмотрели на данные или отсортировали значения. Ускорить данный процесс в дальнейшем помогут методы idxmin() и idxmax(), которые возвращают индекс минимального или максимального значения. <br>
Сохраните название района с наименьшим числом поездок в переменную min_pickups, применив подходящий метод.<br>
Данные сохранены в переменную taxi.

In [18]:
min_pickups = taxi.groupby('borough').agg({'pickups':'sum'}).idxmin()

Продолжим изучение данных и посмотрим на число поездок в выходные дни. Сгруппируйте данные по двум признакам: району города и является ли день выходным (колонки borough и hday). Сравните среднее число поездок, и выберите районы, из которых по праздникам в среднем поступает больше заказов, чем в обычные дни. 

In [19]:
taxi.head()

,pickup_dt,pickup_month,borough,pickups,hday,spd,vsb,temp,dewp,slp,pcp_01,pcp_06,pcp_24,sd
0,2015-01-01 01:00:00,Jan,Bronx,152,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0
1,2015-01-01 01:00:00,Jan,Brooklyn,1519,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0
2,2015-01-01 01:00:00,Jan,EWR,0,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0
3,2015-01-01 01:00:00,Jan,Manhattan,5258,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0
4,2015-01-01 01:00:00,Jan,Queens,405,Y,5.0,10.0,30.0,7.0,1023.5,0.0,0.0,0.0,0.0


In [20]:
taxi.groupby(['borough','hday'], as_index=False).agg({'pickups':'mean'})

,borough,hday,pickups
0,Bronx,N,50.771073
1,Bronx,Y,48.065868
2,Brooklyn,N,534.727969
3,Brooklyn,Y,527.011976
4,EWR,N,0.023467
5,EWR,Y,0.041916
6,Manhattan,N,2401.302921
7,Manhattan,Y,2035.928144
8,Queens,N,308.899904
9,Queens,Y,320.730539


In [21]:
# EWR, Queens

Для каждого района посчитайте число поездок по месяцам. Отсортируйте полученные значения по убыванию и сохраните результирующий датафрейм в pickups_by_mon_bor. 

In [22]:
pickups_by_mon_bor = taxi \
    .groupby(['pickup_month','borough'], as_index=False) \
    .agg({'pickups':'sum'}) \
    .sort_values('pickups', ascending=False)

Напишите функцию temp_to_celcius, которая получает на вход колонку с температурой в °F, и возвращает значения, переведенные в градусы Цельсия.

In [23]:
def temp_to_celcius(els):
    new_els = []
    for i in els:
        cel = ((i - 32)*5)/9
        new_els.append(cel)
    return pd.Series(new_els)